In [ ]:
%cd ../
%pip install -e .
%cd tutorials

Import the image from the LARD dataset and plot the 4 corner of the runway

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from poseidon.numpy import generate_rotation_matrix_with_angles

img = mpimg.imread("LARD_example.jpg")
plt.imshow(img)

df = pd.read_csv("extract_labeling_Bing.csv", delimiter=";")
dic = df.to_dict(orient="records")[0]

TL = [dic["x_TL"], dic["y_TL"]]
plt.scatter(TL[0], TL[1], label="Top Left")

TR = [dic["x_TR"], dic["y_TR"]]
plt.scatter(TR[0], TR[1], label="Top Right")

BL = [dic["x_BL"], dic["y_BL"]]
plt.scatter(BL[0], BL[1], label="Bottom Left")

BR = [dic["x_BR"], dic["y_BR"]]
plt.scatter(BR[0], BR[1], label="Bottom Right")

points_2D_LARD = torch.tensor([TL, TR, BL, BR], dtype=torch.float64).unsqueeze(0)
points_2D_LARD_np = points_2D_LARD.squeeze(0).numpy()
print("Points 2D:\n", points_2D_LARD)

plt.legend()
plt.show()

- Compute the rotation matrix (given for this image, the one we want to re-estimate) : from the yaw pitch and rool  
- Recover the position matrix (given for this image, the one we want to re-estimate) : with the previous funtion  
- Create the intraseca matrix : with the parameters given for this image

In [ ]:
# Rotation
yaw, pitch, roll = dic["yaw"], dic["pitch"], dic["roll"]
print("(yaw,pitch,roll):", yaw, pitch, roll)
R = torch.tensor(
    generate_rotation_matrix_with_angles(yaw, pitch, roll), dtype=torch.float64
).unsqueeze(0)
print("Rotation matrix (R):")
print(R, R.shape)


# Position
C = torch.tensor(
    [[dic["lon_cam"]], [dic["lat_cam"]], [dic["alt_cam"]]], dtype=torch.float64
).unsqueeze(0)
print("Camera position (C):", C, C.shape)


# Intraseca
A = torch.tensor([[60, 0, 0], [0, 60, 0], [0, 0, 1]], dtype=torch.float64).unsqueeze(0)
A_np = A.squeeze(0).numpy()
print("Camera intrinsic parameters (A):")
print(A, A.shape)
print(A_np, A_np.shape)

Import the 3D coordinates of the corners :  

In [ ]:
import json

with open("runway_points.json", "r") as f:
    data = json.load(f)

In [ ]:
import numpy as np
import torch

# Récupère les positions des points A, B, C, D

positions_lat = []

for key in ["B", "A", "C", "D"]:
    coordinate = data["CYEG"]["20"][key]["coordinate"]
    positions_lat.append([coordinate["longitude"], coordinate["latitude"], coordinate["altitude"]])


# Conversion en numpy array et torch tensor
points_3D_np = np.array(positions_lat)

points_3D = torch.tensor(points_3D_np, dtype=torch.float64).unsqueeze(0)

print("3D points (numpy):\n", points_3D_np, points_3D_np.shape)
print("3D points (torch):\n", points_3D, points_3D.shape)

from poseidon.torch.utils.before_p3p import projection_all_point3D_to2D

points_2D = projection_all_point3D_to2D(points_3D, C, R, A)
points_2D_np = points_2D.squeeze(0).numpy()
print("2D points (torch) : \n", points_2D, points_2D.shape)
print("2D", points_2D_LARD)

In [ ]:
# Create of the 3D figure
print("3D points (torch):\n", points_3D, points_3D.shape)
print("Camera position (C):", C, C.shape)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(C[:, 0].squeeze(), C[:, 1].squeeze(), C[:, 2].squeeze(), color="orange")


for i in range(4):
    Pi = points_3D[:, i].squeeze()
    print(f"P{i+1} position:", Pi)
    ax.scatter(*Pi, color="black")
    ax.text(*Pi, f"$P_{i+1}$")
    ax.plot(
        [C[:, 0].squeeze(), Pi[0]], [C[:, 1].squeeze(), Pi[1]], [C[:, 2].squeeze(), Pi[2]], "k--"
    )
    f = Pi - C

# Set axis limits to visualize a 4x4x4 space centered around zero

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("The three 3D Points use for P3P")
ax.view_init(elev=15, azim=30)  # élévation de 60°, azimut de 30°

ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import torch
from poseidon.torch import compute_features_vectors

features_vect = compute_features_vectors(points_3D, C, R)
print("Feature vectors:\n", features_vect, features_vect.shape)


import torch


def compute_feature_vectors_from_2D(points_2D, A):
    fet = []

    fx = A[0, 0, 0]
    fy = A[0, 1, 1]
    cx = A[0, 0, 2]
    cy = A[0, 1, 2]

    print("fx, fy, cx, cy:", fx, fy, cx, cy)

    for i in range(3):
        # points_2D shape: [1, 3, 2], so points_2D[0, i] is [u, v]
        u, v = points_2D[0, i]

        # Normalize using intrinsics
        f = torch.tensor([(u - cx) / fx, (v - cy) / fy, 1.0], dtype=torch.float64)

        # Normalize vector to make it unitary
        f = f / torch.linalg.norm(f)

        fet.append(f)

    return torch.stack(fet, dim=0)  # Shape: [3, 3]


ft = compute_feature_vectors_from_2D((points_2D_LARD[:, :3]), A).unsqueeze(0)
# print("Feature vectors from 2D points:\n", ft)

In [ ]:
# Create of the 3D figure
print("3D points (torch):\n", points_3D, points_3D.shape)
print("Camera position (C):", C, C.shape)
print("Feature vectors (ft):", ft, ft.shape)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(C[:, 0].squeeze(), C[:, 1].squeeze(), C[:, 2].squeeze(), color="orange")
origin = np.array([C[:, 0].item(), C[:, 1].item(), C[:, 2].item()])


ft_sq = ft.squeeze(0).numpy()

for i in range(3):
    Pi = points_3D[:, i].squeeze()
    print(f"P{i+1} position:", Pi)
    ax.scatter(*Pi, color="black")
    ax.text(*Pi, f"$P_{i+1}$")
    ax.plot(
        [C[:, 0].squeeze(), Pi[0]], [C[:, 1].squeeze(), Pi[1]], [C[:, 2].squeeze(), Pi[2]], "k--"
    )
print("ft_sq[:,0]:", ft_sq[:, 0])  # x composantes des vecteurs
print("ft_sq[:,1]:", ft_sq[:, 1])  # y composantes
print("ft_sq[:,2]:", ft_sq[:, 2])  # z composantes
print("origin:", origin)

scale = 1

ax.quiver(
    C[:, 0].item(),
    C[:, 1].item(),
    C[:, 2].item(),
    ft_sq[:, 0] * scale,
    ft_sq[:, 1] * scale,
    ft_sq[:, 2] * scale,
    color=["r", "g", "b"],
)


# Set axis limits to visualize a 4x4x4 space centered around zero

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("The three 3D Points use for P3P")

ax.set_xlim(C[:, 0].item() - 1, C[:, 0].item() + 1)
ax.set_ylim(C[:, 1].item() - 1, C[:, 1].item() + 1)
ax.set_zlim(C[:, 2].item() - 40, C[:, 2].item() + 1)


ax.grid(True)
plt.tight_layout()
plt.show()

Test with open cv : no solution

In [ ]:
from poseidon.numpy import solve_reformat_p3p_solutions

solutions_opencv = solve_reformat_p3p_solutions(points_3D_np[:3, :], points_2D_LARD_np[:3, :], A_np)
print("Solutions from OpenCV P3P:\n", solutions_opencv)

In [ ]:
from poseidon.torch import P3P, find_best_solution_P3P_batch

solutions_torch = P3P(points_3D, features_vect)
print("Solutions from Poseidon P3P:\n", solutions_torch)


R_solutions, C_solutions, error = find_best_solution_P3P_batch(
    solutions_torch, points_2D, points_3D, A
)
print("Rotation solutions (R):\n", R_solutions)
print("R", R)
print("Camera position solutions (C):\n", C_solutions)
print("C", C)
print("Error for each solution:\n", error)